In [1]:
from torch.utils.data import DataLoader
from datasets import load_from_disk
from src.hyperdas.data_utils import generate_ravel_dataset, get_ravel_collate_fn, filter_dataset

from transformers import AutoTokenizer, LlamaForCausalLM

%load_ext autoreload
%autoreload 2

/nlp/scr/sjd24/miniconda3/envs/hyperdas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("/nlp/scr/sjd24/llama3-8b")
gemma_config = AutoConfig.from_pretrained("/nlp/scr/sjd24/cache/hub/models--google--gemma-2-9b-it/snapshots/11c9b309abf73637e4b6f9a3fa1e92e615547819")

In [ ]:
config.hidden_size = gemma_config.hidden_size
config.intermediate_size = gemma_config.intermediate_size
config.num_attention_heads = gemma_config.num_attention_heads

(config.hidden_size, config.intermediate_size, config.num_attention_heads)

(3584, 14336, 16)

In [9]:
from transformers import AutoModelForCausalLM
llama_customized = AutoModelForCausalLM.from_config(config)

In [10]:
llama_customized

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3584)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (k_proj): Linear(in_features=3584, out_features=1792, bias=False)
          (v_proj): Linear(in_features=3584, out_features=1792, bias=False)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3584,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3584,), eps=1e-05)
      )
    )
    (n

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/nlp/scr/sjd24/llama3-8b")
model = LlamaForCausalLM.from_pretrained("/nlp/scr/sjd24/llama3-8b")

model = model.cuda()

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


testset = generate_ravel_dataset(
    n_samples=5000,
    root_path="./data/RAVEL/",
    domain="verb",
    target_attributes=["Definition"],
    isolate_attributes=["Singular", "Past Tense"],
    entity_split="both",
    template_split="test"
)

for _ in range(2):
     testset = filter_dataset(model, tokenizer, testset, relative_position_ids=True, eval_n_label_tokens=1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.18it/s]
313it [02:00,  2.60it/s]


Accuracy: 0.4752; filtered out 2624 examples


149it [00:37,  3.95it/s]

Accuracy: 0.9949494949494949; filtered out 12 examples


In [ ]:
testset.save_to_disk("./experiments/RAVEL/data/ICLR/verb_definition")

2364